In [ ]:
# ==============================================
# Option Straddle Earnings Strategy Backtest Pipeline
# ----------------------------------------------
# Language: Python
# Description:
#   The goal is to sell at-the-money (ATM) call and put options the day before
#   earnings, capture implied volatility (IV) crush, and close the position
#   immediately after earnings (next market open).
#
# Assumptions & Placeholders:
#   - Option data includes implied vol, realized vol, strike, expiry, etc.
#   - Earnings calendar data lists earnings date for each ticker.
#   - IV and RV are historical averages; placeholders used below.
#   - Data sources: 
#   - Real-world implementation requires risk management & margin modelling.
# ==============================================

import pandas as pd
import numpy as np

In [ ]:
# ----------------------------------------------
# STEP 1: Data Loading (placeholders)
# ----------------------------------------------
# Placeholder CSV paths — replace with actual file paths or API queries.
# Each dataset must include columns consistent with comments below.

# Example placeholder structure for options data
# Columns: ['date', 'ticker', 'expiry', 'strike', 'option_type', 'implied_vol', 'price', 'underlying_price']
options_data_path = "./data/options_data.csv"

# Example placeholder structure for earnings data
# Columns: ['ticker', 'earnings_date']
earnings_data_path = "./data/earnings_calendar.csv"

# Example placeholder structure for realized volatility (historical window)
# Columns: ['ticker', 'date', 'realized_vol']
realized_vol_path = "./data/realized_vol.csv"

# Load CSVs (if they exist)
try:
    options_df = pd.read_csv(options_data_path, parse_dates=['date', 'expiry'])
    earnings_df = pd.read_csv(earnings_data_path, parse_dates=['earnings_date'])
    realized_vol_df = pd.read_csv(realized_vol_path, parse_dates=['date'])
except FileNotFoundError:
    # Use empty placeholders to allow code to run syntactically
    options_df = pd.DataFrame()
    earnings_df = pd.DataFrame()
    realized_vol_df = pd.DataFrame()


In [ ]:
# ----------------------------------------------
# STEP 2: Preprocessing / Alignment
# ----------------------------------------------
# Merge or align realized vol with options implied vol for comparison.

# Placeholder: Merge on ticker + date (you'll likely use rolling RV windows)
merged_df = pd.merge(options_df, realized_vol_df, on=['ticker', 'date'], how='left')

In [ ]:
# ----------------------------------------------
# STEP 3: Entry Criteria
# ----------------------------------------------
# Entry rule summary:
#   1. Option is ATM (strike ≈ underlying price)
#   2. Trade occurs 1 trading day before earnings_date
#   3. Implied Volatility (IV) > Realized Volatility (RV) threshold
#   4. Optional: minimum liquidity or open interest filter

# Placeholder parameters
ATM_TOLERANCE = 0.01   # 1% difference between strike and spot
IV_RV_THRESHOLD = 1.2   # IV must be 20% higher than historical RV

# Placeholder: join with earnings calendar to find pre-earnings day entries
if not earnings_df.empty:
    # Merge option data with nearest earnings date per ticker
    merged_df = pd.merge_asof(
        merged_df.sort_values('date'),
        earnings_df.sort_values('earnings_date'),
        left_on='date',
        right_on='earnings_date',
        by='ticker',
        direction='forward'
    )

# Filter for entry day (1 day before earnings)
merged_df['days_to_earnings'] = (merged_df['earnings_date'] - merged_df['date']).dt.days
entries = merged_df.query('days_to_earnings == 1')

# Filter ATM options and IV>RV condition
entries = entries[
    (abs(entries['strike'] / entries['underlying_price'] - 1) < ATM_TOLERANCE) &
    (entries['implied_vol'] > IV_RV_THRESHOLD * entries['realized_vol'])
]

In [ ]:

# ----------------------------------------------
# STEP 4: Trade Simulation (Simplified)
# ----------------------------------------------
# Each trade = sell 1 call + 1 put (same strike, expiry = nearest monthly)
# P/L = (premium_collected - post_earnings_value)

# Placeholder assumptions:
# - Option premium = price at entry
# - Post-earnings IV crush reduces option price proportionally
# - Use simple proportional drop model: post_price = pre_price * (IV_after / IV_before)

IV_CRUSH_FACTOR = 0.6  # 40% implied vol drop after earnings

entries['premium_collected'] = entries['price'] * 2  # call + put
entries['post_earnings_price'] = entries['price'] * IV_CRUSH_FACTOR
entries['pnl'] = entries['premium_collected'] - 2 * entries['post_earnings_price']

# Add stop-loss rule (optional placeholder)
STOP_LOSS = -1.0  # -$1 per straddle placeholder
entries['pnl'] = np.where(entries['pnl'] < STOP_LOSS, STOP_LOSS, entries['pnl'])

# ----------------------------------------------
# STEP 5: Aggregate Results
# ----------------------------------------------
# Summarize average and total P&L per ticker and overall

if not entries.empty:
    results_summary = entries.groupby('ticker')['pnl'].agg(['count', 'mean', 'sum']).reset_index()
    print("=== Strategy Results Summary ===")
    print(results_summary.head())
else:
    print("No trades met entry criteria. Check data and assumptions.")


In [ ]:

# ----------------------------------------------
# STEP 6: Extensions / To Implement Later
# ----------------------------------------------
# [ ] Use actual post-earnings option chain to compute true IV crush
# [ ] Include open interest and volume filters
# [ ] Integrate realized P/L with portfolio-level risk metrics (Sharpe, drawdown)
# [ ] Pull real data using APIs (Yahoo Finance, Polygon.io)
# [ ] Add regression to predict IV crush magnitude vs historical volatility spread

# ==============================================
# END OF PIPELINE
# ==============================================
